In [10]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models

# GPU setup
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
print(f"Using device: {device}")

Using device: cuda


In [11]:
BRAILLE_CHAR_SIZE = (28, 28)

def load_dataset(dataset_path):
    images = []
    labels = []
    
    # Define subdirectories to process
    subdirs = ['books', 'handwritten',  'not_braille', 'uploaded']

    
    for subdir in subdirs:
        subdir_path = os.path.join(dataset_path, subdir)
        train_txt_path = os.path.join(subdir_path, 'train.txt')
        
        if not os.path.exists(train_txt_path):
            print(f"Warning: {train_txt_path} not found. Skipping.")
            continue
        
        # Read train.txt to get list of training images
        with open(train_txt_path, 'r') as f:
            train_files = f.read().splitlines()
        
        for file in train_files:
            img_path = os.path.join(subdir_path, file)
            csv_path = os.path.splitext(img_path)[0] + '.csv'
            
            if not os.path.exists(img_path) or not os.path.exists(csv_path):
                print(f"Warning: Image or CSV file not found for {file}. Skipping.")
                continue
            
            try:
                # Load image
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                img = transforms.ToTensor()(img)
                
                # Load CSV annotation
                with open(csv_path, 'r') as f:
                    for line in f:
                        try:
                            left, top, right, bottom, label = map(float, line.strip().split(';'))
                            
                            # Extract Braille character
                            char_img = img[:, int(top*img.shape[1]):int(bottom*img.shape[1]), 
                                           int(left*img.shape[2]):int(right*img.shape[2])]
                            
                            # Resize the Braille character to a fixed size
                            char_img_resized = transforms.Resize(BRAILLE_CHAR_SIZE)(char_img)
                            
                            # Flatten the image for SVM input
                            char_vector = char_img_resized.flatten().numpy()
                            
                            images.append(char_vector)
                            labels.append(int(label))
                        except ValueError:
                            print(f"Warning: Invalid line in {csv_path}. Skipping.")
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")
    
    if not images:
        raise ValueError("No valid images found in the dataset.")
    
    return np.array(images), np.array(labels)

def train_svm_model(X_train, y_train):
    model = svm.SVC(kernel='rbf', C=1.0, random_state=42)
    model.fit(X_train, y_train)
    return model

def train_dnn_model(X_train, y_train, X_val, y_val, input_dim, num_classes):
    # Initialize a Sequential model
    model = models.Sequential()
    
    # Add input layer (with the number of input features as input_dim)
    model.add(layers.Dense(128, activation='relu', input_dim=input_dim))  # First hidden layer
    
    # Add additional hidden layers
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    
    # Output layer for classification
    model.add(layers.Dense(num_classes, activation='softmax'))  # For multi-class classification
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
    
    return model, history


def preprocess_image(image):
    # Convert to grayscale if it's not already
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Convert to PIL Image
    pil_image = Image.fromarray(image)
    
    # Resize to match BRAILLE_CHAR_SIZE
    resized = pil_image.resize(BRAILLE_CHAR_SIZE)
    
    # Convert to tensor and flatten
    tensor = transforms.ToTensor()(resized)
    flattened = tensor.flatten().numpy()
    
    return flattened

# def translate_braille_realtime(model, scaler):
#     cap = cv2.VideoCapture(0)
    
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         # Preprocess the frame
#         processed_frame = preprocess_frame(frame)
        
#         # Scale the processed frame
#         scaled_frame = scaler.transform([processed_frame])
        
#         # Predict using SVM
#         prediction = model.predict(scaled_frame)[0]
        
#         # Convert prediction to Braille character
#         braille_char = chr(0x2800 + prediction)  # Unicode Braille patterns start at U+2800
        
#         # Display result
#         cv2.putText(frame, f"Detected: {braille_char}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#         cv2.imshow('Braille Translator', frame)
        
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
    
#     cap.release()
#     cv2.destroyAllWindows()

# # Main execution
# if __name__ == "__main__":
#     dataset_path = os.path.join(os.getcwd(), "AngelinaDataset-master")
    
#     # Load and preprocess dataset
#     print("Loading dataset...")
#     try:
#         X, y = load_dataset(dataset_path)
        
#         # Split the data
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#         # Normalize the data
#         scaler = StandardScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
#         X_test_scaled = scaler.transform(X_test)
        
#         # Train SVM model
#         print("Training SVM model...")
#         model = train_svm_model(X_train_scaled, y_train)
        
#         # Evaluate the model
#         accuracy = model.score(X_test_scaled, y_test)
#         print(f"Model accuracy: {accuracy:.2f}")
        
#         # Start real-time translation
#         print("Starting real-time translation. Press 'q' to quit.")
#         translate_braille_realtime(model, scaler)
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         import traceback
#         traceback.print_exc()

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [5]:
import tensorflow as tf

# List all available physical devices (this should include GPUs)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Verify the GPU device
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow GPU is configured correctly.")
else:
    print("TensorFlow is not detecting your GPU.")


Num GPUs Available:  0
TensorFlow is not detecting your GPU.


In [6]:
# Train and save the model
dataset_path = os.path.join(os.getcwd(), "AngelinaDataset-master")

print("Loading dataset...")
X, y = load_dataset(dataset_path)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#random state bawaan 42

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM model
print("Training SVM model...")
model = train_svm_model(X_train_scaled, y_train)

# Evaluate the model
accuracy = model.score(X_test_scaled, y_test)
print(f"Model accuracy: {accuracy:.2f}")

# Save the model and scaler
joblib.dump(model, 'braille_svm_model.joblib')
joblib.dump(scaler, 'braille_scaler.joblib')
print("Model and scaler saved.")

Loading dataset...
Training SVM model...
Model accuracy: 0.91
Model and scaler saved.


In [12]:
# Train and save the model
dataset_path = os.path.join(os.getcwd(), "AngelinaDataset-master")

print("Loading dataset...")
X, y = load_dataset(dataset_path)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#random state bawaan 42

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM model
print("Training SVM model...")
model = train_dnn_model(X_train_scaled, y_train)

# Evaluate the model
accuracy = model.score(X_test_scaled, y_test)
print(f"Model accuracy: {accuracy:.2f}")

# Save the model and scaler
joblib.dump(model, 'braille_svm_model.joblib')
joblib.dump(scaler, 'braille_scaler.joblib')
print("Model and scaler saved.")

Loading dataset...
Training SVM model...


TypeError: train_dnn_model() missing 4 required positional arguments: 'X_val', 'y_val', 'input_dim', and 'num_classes'

In [9]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

def load_model_and_scaler():
    model = joblib.load('braille_svm_model.joblib')
    scaler = joblib.load('braille_scaler.joblib')
    return model, scaler

def translate_braille(image, model, scaler):
    processed_image = preprocess_image(image)
    scaled_image = scaler.transform([processed_image])
    prediction = model.predict(scaled_image)[0]
    return chr(0x2800 + prediction)

def realtime_translation():
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Translate the frame
        braille_char = translate_braille(frame, model, scaler)
        
        # Display result
        cv2.putText(frame, f"Detected: {braille_char}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Braille Translator', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        braille_char = translate_braille(image, model, scaler)
        result_label.config(text=f"Detected Braille: {braille_char}")
        
        # Display the image
        pil_image = Image.open(file_path)
        pil_image.thumbnail((300, 300))  # Resize for display
        tk_image = ImageTk.PhotoImage(pil_image)
        image_label.config(image=tk_image)
        image_label.image = tk_image  # Keep a reference

# Load the trained model and scaler
model, scaler = load_model_and_scaler()

# Create the main window
root = tk.Tk()
root.title("Braille Translator")

# Create and pack widgets
tk.Button(root, text="Real-time Translation", command=realtime_translation).pack(pady=10)
tk.Button(root, text="Upload Image", command=upload_image).pack(pady=10)
result_label = tk.Label(root, text="")
result_label.pack(pady=10)
image_label = tk.Label(root)
image_label.pack(pady=10)

# Start the GUI event loop
root.mainloop()